In [1]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pickle

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [12]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [13]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [14]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks +=['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [15]:
with open('coordinates.csv', mode="w", newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
class_name = "hello"

In [8]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
                
            row = pose_row+face_row
            row.insert(0, class_name)
            
            with open('coordinates.csv', mode="a", newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except: 
            pass
        
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [9]:
df = pd.read_csv('coordinates.csv')

In [10]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,smile,0.594136,0.940849,-1.385004,1.0,0.611982,0.846764,-1.376439,1.0,0.631851,...,-0.028847,0.0,0.676093,0.834288,0.003040,0.0,0.682117,0.821155,0.003873,0.0
1,smile,0.598051,0.951782,-1.641750,1.0,0.622456,0.848134,-1.644646,1.0,0.646696,...,-0.032081,0.0,0.673553,0.827806,-0.004173,0.0,0.679734,0.816454,-0.003818,0.0
2,smile,0.609512,0.949932,-2.091486,1.0,0.633684,0.837606,-2.099624,1.0,0.660521,...,-0.033710,0.0,0.669326,0.815230,-0.008987,0.0,0.675483,0.803972,-0.009107,0.0
3,smile,0.606072,0.958241,-2.424640,1.0,0.632875,0.845127,-2.436252,1.0,0.660524,...,-0.030536,0.0,0.669198,0.822180,-0.007269,0.0,0.675529,0.808064,-0.006900,0.0
4,smile,0.595578,0.963908,-2.226414,1.0,0.625129,0.854036,-2.243140,1.0,0.652389,...,-0.032529,0.0,0.669595,0.821507,-0.008805,0.0,0.675662,0.807390,-0.008446,0.0


In [11]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
2,smile,0.609512,0.949932,-2.091486,1.0,0.633684,0.837606,-2.099624,1.0,0.660521,...,-0.033710,0.0,0.669326,0.815230,-0.008987,0.0,0.675483,0.803972,-0.009107,0.0
3,smile,0.606072,0.958241,-2.424640,1.0,0.632875,0.845127,-2.436252,1.0,0.660524,...,-0.030536,0.0,0.669198,0.822180,-0.007269,0.0,0.675529,0.808064,-0.006900,0.0
4,smile,0.595578,0.963908,-2.226414,1.0,0.625129,0.854036,-2.243140,1.0,0.652389,...,-0.032529,0.0,0.669595,0.821507,-0.008805,0.0,0.675662,0.807390,-0.008446,0.0
5,smile,0.580029,0.967034,-2.514566,1.0,0.610741,0.856785,-2.531772,1.0,0.639317,...,-0.033269,0.0,0.665288,0.813109,-0.010087,0.0,0.671429,0.799794,-0.009994,0.0
6,smile,0.593939,0.951779,-2.385121,1.0,0.620132,0.838604,-2.393335,1.0,0.647745,...,-0.030293,0.0,0.661277,0.799122,-0.005793,0.0,0.667083,0.789079,-0.005820,0.0


In [25]:
df[df['class'] == 'shinchan']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501


In [26]:
x = df.drop('class', axis=1)
y = df['class']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [28]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [29]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [30]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [31]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.9791666666666666


In [32]:
fit_models['rf'].predict(x_test)

array(['smile', 'smiling', 'smiling', 'smile', 'smile', 'smile', 'smile',
       'smile', 'smiling', 'smiling', 'smiling', 'smile', 'smile',
       'smile', 'smile', 'smiling', 'smile', 'smiling', 'smiling',
       'smile', 'smiling', 'smiling', 'smile', 'smiling', 'smiling',
       'smiling', 'smile', 'smile', 'smile', 'smiling', 'smile', 'smile',
       'smile', 'smile', 'smile', 'smile', 'smile', 'smile', 'smiling',
       'smile', 'smile', 'smile', 'smiling', 'smile', 'smiling', 'smile',
       'smile', 'smiling'], dtype=object)

In [33]:
y_test

94       smile
27     smiling
57     smiling
132      smile
73       smile
121      smile
74       smile
151      smile
39     smiling
6      smiling
25     smiling
119      smile
79       smile
63       smile
117      smile
61     smiling
109      smile
9      smiling
35     smiling
136      smile
29     smiling
58     smiling
105      smile
40     smiling
42     smiling
11     smiling
123      smile
103      smile
147      smile
17     smiling
137      smile
92       smile
65       smile
99       smile
149      smile
125      smile
157      smile
142      smile
51     smiling
129      smile
88       smile
93       smile
48     smiling
97       smile
43     smiling
111      smile
134      smile
54     smiling
Name: class, dtype: object

In [34]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [35]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [36]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.01 0.99]
smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.07 0.93]
smiling [0.05 0.95]
smiling [0.02 0.98]
smiling [0.01 0.99]
smiling [0.01 0.99]
smiling [0. 1.]
smiling [0. 1.]
smiling [0.03 0.97]
smiling [0. 1.]
smiling [0. 1.]
smiling [0. 1.]
smiling [0.01 0.99]
smiling [0.01 0.99]
smiling [0.06 0.94]
smiling [0.06 0.94]
smiling [0.35 0.65]
smiling [0.14 0.86]
smiling [0.08 0.92]
smiling [0.07 0.93]
smiling [0.08 0.92]
smiling [0.08 0.92]
smiling [0.07 0.93]
smiling [0.07 0.93]
smiling [0.07 0.93]
smiling [0.07 0.93]
smiling [0.08 0.92]
smiling [0.09 0.91]
smiling [0.09 0.91]
smiling [0.09 0.91]
smiling [0.08 0.92]
smiling [0.08 0.92]
smiling [0.09 0.91]
smiling [0.09 0.91]
smiling [0.09 0.91]
smiling [0.06 0.94]
smiling [0.04 0.96]
smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.03 0.97]
smiling [0.02 0.98]
smiling [0.02 0.98]
smiling [0.02 0.98]
smiling [0.02 0.98]
smiling [0.03 0.97]


In [183]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(356, 151)